<a href="https://colab.research.google.com/github/ZamanRepo/ADiffP/blob/master/py_Memory_Profiler_Tutorial_Zaman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://builtin.com/data-science/python-profiling-tools-tutorial

@author: sadrachpierre


In [7]:
pip install memory_profiler


In [9]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.5 MB/s eta 0:00:00


In [10]:
from memory_profiler import profile

In [38]:
from memory_profiler import profile
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score

from catboost import CatBoostClassifier

from timeit import default_timer as timer

import cProfile, pstats, io
from pstats import SortKey

To start, let’s import the Pandas library and read our data into a Pandas data frame:

In [12]:
df = pd.read_csv("/content/drive/MyDrive/AI-CS-Nov2023/creditcard.csv")


# Next, let’s relax the display limits for columns and rows using the Pandas method set_option():



In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Next, let’s display the first five rows of data using the head() method:


In [26]:
print(df.head())

            Time        V1        V2        V3        V4        V5        V6  \
123719   77035.0 -0.307394  1.054983  0.940188 -0.120699  0.435110 -0.535121   
209769  137697.0  0.271729  1.124104 -1.705547 -0.336154  0.863448 -1.206388   
38837    39573.0 -1.204608  1.619422  0.842481  1.626251 -0.101059 -0.005622   
38953    39628.0 -1.419886  1.808589  0.457963  2.471223  0.183311  1.305300   
99062    66942.0 -0.208778  1.420082 -0.108277  0.989850  0.126301 -1.161187   

              V7        V8        V9       V10       V11       V12       V13  \
123719  0.663844 -0.005949 -0.300109 -0.502034 -1.000976 -0.477544  0.031318   
209769  0.812369  0.108942 -0.305962 -1.069973 -0.992622 -0.196218 -0.233032   
38837   0.011532  0.858761 -0.950160 -0.435792 -1.917637  0.497064  0.745681   
38953  -0.093348  0.953873 -2.068809  0.616840  0.241472  0.640245  1.302956   
99062   0.768593  0.090921 -0.873187 -0.063377 -0.484934  0.734898  1.190559   

             V14       V15       V16  

Next, to get an idea of how big this data set is, we can use the len method to see how many rows there are:

And we can do something similar for counting the number of columns. We can assess the columns attribute from our Pandas data frame object and use the len() method to count the number of columns:




In [16]:
print("Number of rows: ", len(df))
print("Number of columns: ", len(df.columns))


Number of rows:  284807
Number of columns:  31


# We can see that this data set is relatively large: 284,807 rows and 31 columns. Further, it takes up 150 MB of space. To demonstrate the benefits of profiling in Python, we’ll start with a small subsample of this data on which we’ll perform ETL and train a classification model.

# Let’s proceed by generating a small subsample data set. Let’s take a random sample of 10,000 records from our data. We will also pass a value for random_state, which will guarantee that we select the same set of records every time we run the script. We can do this using the sample() method on our Pandas data frame:

In [27]:
df = df.sample(10000, random_state=42)


Next, we can write the subsample of our data to a new csv file:

In [28]:
df.to_csv("creditcard_subsample10000.csv", index=False)

In [97]:
@profile
def read_data(filename):
    df = pd.read_csv(filename)
    return df

In [113]:
@profile
def data_prep(dataframe, columns):
    df_select = dataframe[columns]
    return df_select

In [114]:
@profile
def feature_engineering(dataframe, inputs, output):
    X = dataframe[inputs]
    y = dataframe[output]
    return X, y

In [115]:
@profile
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.33)
    return X_train, X_test, y_train, y_test

In [116]:
@profile
def model_training(X_train, y_train, model_type):
    if model_type == 'Logistic Regression':
        model = LogisticRegression()
        model.fit(X_train, y_train)
    elif model_type == 'CatBoost':
        model = CatBoostClassifier()
        model.fit(X_train, y_train)
    return model

In [117]:
@profile
def predict(model, X_test):
    y_pred = model.predict(X_test)
    return y_pred

In [118]:
@profile
def evaluate(y_pred, y_test):
    precision = average_precision_score(y_test, y_pred)
    print("Precision: ", precision)

In [138]:
def main():
    runtime_metrics = dict()
    #read in data
    start = timer()
    data = read_data('/content/drive/MyDrive/AI-CS-Nov2023/creditcard.csv')
    end = timer()
    read_time = end - start
    runtime_metrics['read_time'] = read_time

In [140]:
#slect relevant columns
start = timer()
columns = ['V1', 'V2', 'V3', 'Amount', 'Class']
df_select = data_prep(data, columns)
end = timer()
select_time = end - start
runtime_metrics['select_time'] = select_time

NameError: ignored

In [135]:
from pandas.io.parsers.readers import read_csv
def main():
    runtime_metrics = dict()
    #read in data
    start = timer()
    data = read_data('/content/drive/MyDrive/AI-CS-Nov2023/creditcard.csv')
    end = timer()
    read_time = end - start
    runtime_metrics['read_time'] = read_time

    #slect relevant columns
    start = timer()
    columns = ['V1', 'V2', 'V3', 'Amount', 'Class']
    df_select = data_prep(data, columns)
    end = timer()
    select_time = end - start
    runtime_metrics['select_time'] = select_time

In [136]:
#slect relevant columns
start = timer()
columns = ['V1', 'V2', 'V3', 'Amount', 'Class']
df_select = data_prep(data, columns)
end = timer()
select_time = end - start
runtime_metrics['select_time'] = select_time

NameError: ignored

In [120]:
#define input and output
start = timer()
inputs = ['V1', 'V2', 'V3']
output = 'Class'
X, y = feature_engineering(df_select, inputs, output)
end = timer()
data_prep_time = end - start
runtime_metrics['data_prep_time'] = data_prep_time

NameError: ignored

### Defining Our ML Pipeline
Now we can start building out the logic for data preparation and model training.

---


Let’s define a method that reads in our csv file, stores it in a data frame and returns it:

In [30]:
def read_data(filename):
    df = pd.read_csv(filename)
    return df

Next, let’s define a function that selects a subset of columns in the data. The function will take a data frame and a list of columns as inputs and return a new one with the selected columns:

In [31]:
def data_prep(dataframe, columns):
    df_select = dataframe[columns]
    return df_select

Next, let’s define a method that itself defines model inputs and outputs and returns these values:

In [32]:
def feature_engineering(dataframe, inputs, output):
    X = dataframe[inputs]
    y = dataframe[output]
    return X, y

We can then define a method used for splitting data for training and testing. First, at the top of our script, let’s import the train_test_split method from the model_selection module in Scikit-learn:

In [39]:
from sklearn.model_selection import train_test_split

Now we can define our method for splitting our data:

In [40]:
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y,
random_state=42, test_size = 0.33)
    return X_train, X_test, y_train, y_test

In [59]:
pip install scikit-learn

In [65]:
#optional added by Zaman
!apt install sklearn.linear_models

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package sklearn.linear_models


In [69]:
#optional added by Zaman
import sklearn;
sklearn.show_versions()


System:
    python: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
executable: /usr/bin/python3
   machine: Linux-5.15.120+-x86_64-with-glibc2.35

Python dependencies:
      sklearn: 1.2.2
          pip: 23.1.2
   setuptools: 67.7.2
        numpy: 1.23.5
        scipy: 1.11.3
       Cython: 3.0.4
       pandas: 1.5.3
   matplotlib: 3.7.1
       joblib: 1.3.2
threadpoolctl: 3.2.0

Built with OpenMP: True

threadpoolctl info:
       user_api: blas
   internal_api: openblas
    num_threads: 2
         prefix: libopenblas
       filepath: /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-742d56dc.3.20.so
        version: 0.3.20
threading_layer: pthreads
   architecture: Haswell

       user_api: openmp
   internal_api: openmp
    num_threads: 2
         prefix: libgomp
       filepath: /usr/local/lib/python3.10/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
        version: None

       user_api: blas
   internal_api: openblas
    num_threads: 2
        

Next, we can define a method that will fit a model of our choice to our training data. Let’s start with a simple logistic regression model. We can import the logistic regression class from the linear models module in Scikit-learn:

In [74]:
#from sklearn.linear_models import logistic_regression #error
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

We will then define a method that takes our training data and an input that specifies the model type. The model type parameter we will use to define and train a more complex model later on:

In [85]:
def model_training(X_train, y_train, model_type):
    if model_type == 'Logistic Regression':
        model = LogisticRegression()
        model.fit(X_train, y_train)
    elif model_type == 'CatBoost':
        model = CatBoostClassifier()
        model.fit(X_train, y_train)
    return model

Next, we can define a method that take our trained model and test data as inputs and returns predictions:

In [86]:
def predict(model, X_test):
    y_pred = model.predict(X_test)
    return y_pred

Finally, let’s define a method that evaluates our predictions. We’ll use average precision, which is a useful performance metric for imbalance classification problems. An imbalance classification problem is one where one of the targets has significantly fewer examples than the other target(s). In this case, most of the transaction data correspond to legitimate transactions, whereas a small minority of transactions are fraudulent:

In [87]:
def evaluate(y_pred, y_test):
    precision = average_precision_score(y_test, y_pred)
    print("Precision: ", precision)

Now we have all of the logic in place for our simple ML pipeline. Let’s execute this logic for our small subsample of data. First, let’s define a main function that we’ll use to execute our code. In this main function, we’ll read in our subsampled data:

In [89]:
def main():
    data = read_data('creditcard_subsample10000.csv')

Next, use the data prep method to select our columns. Let’s select V1, V2, V3, Amount and Class:

In [90]:
def main():
    ... # preceding code left out for clarity
    columns = ['V1', 'V2', 'V3', 'Amount', 'Class']
    df_select = data_prep(data, columns)

Let’s then define inputs and output. We will use V1, V2, V3, and Amount as inputs; the class will be the output:

In [91]:
def main():
    ... # preceding code left out for clarity
    inputs = ['V1', 'V2', 'V3']
    output = 'Class'
    X, y = feature_engineering(df_select, inputs, output)

We’ll split our data for training and testing:

In [92]:
def main():
    ... # preceding code left out for clarity
    X_train, X_test, y_train, y_test = split_data(X, y)

Fit our data:

In [93]:
def main():
    ... # preceding code left out for clarity
    model_type = 'Logistic Regression'
    model = model_training(X_train, y_train, model_type)

Make predictions:

In [94]:
def main():
    ... # preceding code left out for clarity
    y_pred = predict(model, X_test)

And, finally, evaluate model predictions:

In [95]:
def main():
    ... # preceding code left out for clarity
    evaluate(y_pred, y_test)

We can then execute the main function with the following logic:

In [96]:
if __name__ == "__main__":
    main()

NameError: ignored

In [23]:
@profile
def read_data(filename):
    df = pd.read_csv(filename)
    return df


In [24]:
print(df.head())

            Time        V1        V2        V3        V4        V5        V6  \
123719   77035.0 -0.307394  1.054983  0.940188 -0.120699  0.435110 -0.535121   
209769  137697.0  0.271729  1.124104 -1.705547 -0.336154  0.863448 -1.206388   
38837    39573.0 -1.204608  1.619422  0.842481  1.626251 -0.101059 -0.005622   
38953    39628.0 -1.419886  1.808589  0.457963  2.471223  0.183311  1.305300   
99062    66942.0 -0.208778  1.420082 -0.108277  0.989850  0.126301 -1.161187   

              V7        V8        V9  ...       V21       V22       V23  \
123719  0.663844 -0.005949 -0.300109  ... -0.317828 -0.863688 -0.106087   
209769  0.812369  0.108942 -0.305962  ...  0.321285  0.953738 -0.106337   
38837   0.011532  0.858761 -0.950160  ...  0.123273  0.204256 -0.073849   
38953  -0.093348  0.953873 -2.068809  ...  0.280318  0.568673 -0.271461   
99062   0.768593  0.090921 -0.873187  ...  0.201966  0.688611  0.090539   

             V24       V25       V26       V27       V28  Amount  Cl